In [5]:
import re

def extract_entities(question):
    # Regular expression pattern to match entities (e.g., "Naruto")
    pattern = r'\b\w+\b'
    
    # Find all matches of the pattern in the question
    entities = re.findall(pattern, question)
    
    return entities

# The last question
question = "I just watched a movie called Naruto, can you recommend some similar movie?"

entities = extract_entities(question)
print(entities)

['I', 'just', 'watched', 'a', 'movie', 'called', 'Naruto', 'can', 'you', 'recommend', 'some', 'similar', 'movie']


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances
import rdflib
from rdflib import Graph, URIRef, Literal
import re
import editdistance
import pickle
import os
import time
import llama
import csv
import subprocess
import numpy as np
import sentence_BERT


# def find_label(s):
#     label_space = 'http://www.w3.org/2000/01/rdf-schema#label'
#     query_template_label = "SELECT DISTINCT ?x WHERE {{<{}> <{}> ?x.}}".format(s, label_space)
    
#     qres = g.query(query_template_label)
#     ass = 'Label not found'
#     for row in qres:
#         ass = str(row.x)
#         break
#     return ass

g = Graph()
g.parse('/home/tongsu/Speakeasy-agent/data/14_graph.nt', format='turtle')

node_labels = {}
predicate_labels = {}
path1 = '/home/tongsu/Speakeasy-agent/data/node_labels.pkl'
path2 = '/home/tongsu/Speakeasy-agent/data/predicate_labels.pkl'
with open(path1, 'rb') as f:
    node_labels = pickle.load(f)
with open(path2, 'rb') as f:
    predicate_labels = pickle.load(f)


def template_1(question):
    # # process input
    # sent = process_input(question)
    # predicted_labels = predict_entities(sent, crf)

    # print("Input Text:", question)

    # entity = extract_entities(sent, predicted_labels)

    # entity = ' '.join(entity)

    # #defining a question pattern
    # question_pattern = "who is the (.*) of ENTITY"

    # question = re.sub(entity, "ENTITY", question.rstrip("?"))  # preprocess the question

    # relation = re.match(question_pattern, question).group(1)  # match the relation using a pattern
    
    entity, relation = llama.entity_relation_extract(question)
    
    print("!entity:! ", entity)
    print("!relation:! ", relation)
    
    relation = sentence_BERT.get_closest(relation)
    
    print(relation)

    #匹配
    tmp = 9999
    match_node = ""
    for key, label in node_labels.items():
        if editdistance.eval(label, entity) < tmp:
            tmp = editdistance.eval(label, entity)
            match_node = key

    tmp = 9999
    match_pred = ""
    for key, label in predicate_labels.items():
        if editdistance.eval(label, relation) < tmp:
            tmp = editdistance.eval(label, relation)
            match_pred = key

    # print("\n--- the matching node of \"{}\" is {}\n".format(entity, match_node))
    # print("--- the matching predicates of \"{}\" is {}\n".format(relation, match_pred))

    #query
    query_template = "SELECT DISTINCT ?x ?y WHERE {{ <{}> <{}> ?x.  }}".format(match_node, match_pred )

    qres = g.query(query_template)
    # print("\n--- querying results: ")
    answer = 'oops no answer'
    for row in qres:
        # print(row.x)
        print(type(row.x.toPython()))
        try:
            answer = node_labels[row.x.toPython()]
        except KeyError:
            answer = row.x
    # print("answer is:",format(answer))



    WD = rdflib.Namespace('http://www.wikidata.org/entity/')
    WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
    DDIS = rdflib.Namespace('http://ddis.ch/atai/')
    RDFS = rdflib.namespace.RDFS
    SCHEMA = rdflib.Namespace('http://schema.org/')
    # load the embeddings
    
    entity_emb = np.load("/home/tongsu/Speakeasy-agent/data/embeddings/entity_embeds.npy")
    relation_emb = np.load("/home/tongsu/Speakeasy-agent/data/embeddings/relation_embeds.npy")
    entity_file = "/home/tongsu/Speakeasy-agent/data/embeddings/entity_ids.del"
    relation_file = "/home/tongsu/Speakeasy-agent/data/embeddings/relation_ids.del"
    # load the dictionaries
    with open(entity_file, 'r') as ifile:
        ent2id = {rdflib.term.URIRef(ent): int(idx) for idx, ent in csv.reader(ifile, delimiter='\t')}
        id2ent = {v: k for k, v in ent2id.items()}
    with open(relation_file, 'r') as ifile:
        rel2id = {rdflib.term.URIRef(rel): int(idx) for idx, rel in csv.reader(ifile, delimiter='\t')}
        id2rel = {v: k for k, v in rel2id.items()}
    ent2lbl = {ent: str(lbl) for ent, lbl in g.subject_objects(RDFS.label)}
    lbl2ent = {lbl: ent for ent, lbl in ent2lbl.items()}

    # Find the Wikidata ID for the movie (https://www.wikidata.org/wiki/Q132863 is the ID for "Finding Nemo")
    movie = WD[match_node.split('/')[-1]]

    # Find the movie in the graph
    movie_id = ent2id[movie]

    # we compare the embedding of the query entity to all other entity embeddings
    distances = pairwise_distances(entity_emb[movie_id].reshape(1, -1), entity_emb, metric='cosine').flatten()

    # and sort them by distance
    most_likely = distances.argsort()

    for rank, idx in enumerate(most_likely[:3]):
        rank = rank + 1
        ent = id2ent[idx]
        q_id = ent.split('/')[-1]
        lbl = ent2lbl[ent]
        dist = distances[idx]

    movie_emb = entity_emb[ent2id[movie]]

    try:
        # Find the predicate (relation) of the genre (https://www.wikidata.org/wiki/Property:P136 is the ID for "genre")
        genre = WDT[match_pred.split('/')[-1]]
        genre_emb = relation_emb[rel2id[genre]]

        # combine according to the TransE scoring function
        lhs = movie_emb + genre_emb

        # compute distance to *any* entity
        distances = pairwise_distances(lhs.reshape(1, -1), entity_emb).reshape(-1)

        # find most plausible tails
        most_likely = distances.argsort()

        embedding_list = []

        # show most likely entities
        for rank, idx in enumerate(most_likely[:3]):
            rank = rank + 1
            ent = id2ent[idx]
            q_id = ent.split('/')[-1]
            lbl = ent2lbl[ent]
            dist = distances[idx]
            embedding_list.append(lbl)
            
        answer_embed = ','.join(embedding_list)
    
    except:
        answer_embed = "oops no answer"

    #answer
    if (answer != "oops no answer") and (answer_embed != "oops no answer"):
        answer_template = "Hi, the factual answer is: {}, ".format(answer) + "and the answers suggested by embeddings are: {}.".format(answer_embed)
        
    elif answer == "oops no answer":
        answer_template = "Hi, the answers suggested by embeddings are: {}.".format(answer_embed)
    
    elif answer_embed == "oops no answer":
        answer_template = "Hi, the factual answer is: {}.".format(answer)
    
    else:
        answer_template = "Apologies, but there is no corresponding answer in the database for your question."
        

    # print("\n--- generated response: {}".format(answer_template))
    return answer_template

/home/tongsu/miniconda3/envs/promptir/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
command = ["ollama", "serve"]
process = subprocess.Popen(command)
time.sleep(10)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2024/10/26 00:53:46 routes.go:1158: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/home/tongsu/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 

In [3]:
Input = [
         "Who is the director of Star Wars: Episode VI - Return of the Jedi?",
         "Who is the screenwriter of The Masked Gang: Cyprus? ",
         "Who is the director of Good Will Hunting? ",
         "Who directed The Bridge on the River Kwai?", 
         "What is the MPAA film rating of Weathering with You? ",
         "What is the genre of Good Neighbors? ",
         'When was "The Godfather" released? ',
         ]

for i in Input:
    print(template_1(i))

time=2024-10-26T00:54:11.979+02:00 level=INFO source=common.go:49 msg="Dynamic LLM libraries" runners="[rocm_v60102 cpu cpu_avx cpu_avx2 cuda_v11 cuda_v12]"
time=2024-10-26T00:54:11.979+02:00 level=INFO source=gpu.go:221 msg="looking for compatible GPUs"
time=2024-10-26T00:54:13.651+02:00 level=INFO source=types.go:123 msg="inference compute" id=GPU-30b4bb22-5580-74ed-7d46-cc6dc249dbb8 library=cuda variant=v12 compute=8.6 driver=12.4 name="NVIDIA GeForce RTX 3060 Laptop GPU" total="6.0 GiB" available="5.0 GiB"
time=2024-10-26T00:54:13.900+02:00 level=INFO source=sched.go:714 msg="new model will fit in available VRAM in single GPU, loading" model=/home/tongsu/.ollama/models/blobs/sha256-dde5aa3fc5ffc17176b5e8bdc82f587b24b2678c6c66101bf7da77af9f7ccdff gpu=GPU-30b4bb22-5580-74ed-7d46-cc6dc249dbb8 parallel=4 available=5370806272 required="3.7 GiB"
time=2024-10-26T00:54:14.012+02:00 level=INFO source=server.go:105 msg="system memory" total="12.3 GiB" free="7.0 GiB" free_swap="3.6 GiB"
time=

INFO [main] starting c++ runner | tid="140411533201408" timestamp=1729896854
INFO [main] build info | build=10 commit="3a8c75e" tid="140411533201408" timestamp=1729896854
INFO [main] system info | n_threads=8 n_threads_batch=8 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 0 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | RISCV_VECT = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | " tid="140411533201408" timestamp=1729896854 total_threads=16
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="15" port="46513" tid="140411533201408" timestamp=1729896854


llama_model_loader: loaded meta data with 30 key-value pairs and 255 tensors from /home/tongsu/.ollama/models/blobs/sha256-dde5aa3fc5ffc17176b5e8bdc82f587b24b2678c6c66101bf7da77af9f7ccdff (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 3B
llama_model_loader: - kv   6:                               general.tags arr[st

INFO [main] model loaded | tid="140411533201408" timestamp=1729896876


time=2024-10-26T00:54:37.135+02:00 level=INFO source=server.go:626 msg="llama runner started in 23.12 seconds"


[GIN] 2024/10/26 - 00:54:37 | 200 | 24.054150483s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/10/26 - 00:54:37 | 200 |   92.124626ms |       127.0.0.1 | POST     "/api/chat"
!entity:!  Star Wars: Episode VI - Return of the Jedi
!relation:!  director
director
<class 'str'>
Hi, the factual answer is: Richard Marquand, and the answers suggested by embeddings are: Frank Oz,George Lucas,Lawrence Kasdan.
[GIN] 2024/10/26 - 00:54:52 | 200 |  364.312032ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/10/26 - 00:54:52 | 200 |   99.049653ms |       127.0.0.1 | POST     "/api/chat"
!entity:!  The Masked Gang: Cyprus
!relation:!  screenwriter
screenwriter
Hi, the factual answer is: oops no answer, and the answers suggested by embeddings are: Cengiz Küçükayvaz,Murat Aslan,Melih Ekener.
[GIN] 2024/10/26 - 00:54:54 | 200 |   202.07919ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/10/26 - 00:54:54 | 200 |  139.443954ms |       127.0.0.1 | POST     "/api/chat"
!entity:!  Good Will